In [1]:
%matplotlib inline
from astropy.io import ascii
from astropy.table import Table

import sys
import re
import numpy as np
import pylab
import json
import requests

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib  

In [2]:
from api import *

In [3]:
import pandas as pd

In [4]:
c_patch = pd.read_csv('../src_data/one_patch.csv')

In [7]:
ra = c_patch['RAdeg'].iloc[0]
dec = c_patch['DEdeg'].iloc[0]
radius = 50.0/3600.0
constraints = {'nDetections.gt':1}

# strip blanks and weed out blank and commented-out values
columns = """l,b,
gPSFFlux,gPSFFluxErr,gKronFlux, gKronFluxErr,
rPSFFlux, rPSFFluxErr, rKronFlux, rKronFluxErr,
iPSFFlux, iPSFFluxErr, iKronFlux, iKronFluxErr,
zPSFFlux, zPSFFluxErr, zKronFlux, zKronFluxErr,
yPSFFlux, yPSFFluxErr, yKronFlux, yKronFluxErr""".split(',')
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]
results = ps1cone(ra,dec,radius,release='dr2',columns=columns,verbose=True,**constraints, table='stack')
# print first few lines
lines = results.split('\n')
print(len(lines),"rows in results -- first 5 rows:")
print('\n'.join(lines[:6]))

https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr2/stack.csv?nDetections.gt=1&ra=217.8634834331949&dec=66.54466608500267&radius=0.013888888888888888&columns=%5Bl%2Cb%2CgPSFFlux%2CgPSFFluxErr%2CgKronFlux%2CgKronFluxErr%2CrPSFFlux%2CrPSFFluxErr%2CrKronFlux%2CrKronFluxErr%2CiPSFFlux%2CiPSFFluxErr%2CiKronFlux%2CiKronFluxErr%2CzPSFFlux%2CzPSFFluxErr%2CzKronFlux%2CzKronFluxErr%2CyPSFFlux%2CyPSFFluxErr%2CyKronFlux%2CyKronFluxErr%5D
11 rows in results -- first 5 rows:
l,b,gPSFFlux,gPSFFluxErr,gKronFlux,gKronFluxErr,rPSFFlux,rPSFFluxErr,rKronFlux,rKronFluxErr,iPSFFlux,iPSFFluxErr,iKronFlux,iKronFluxErr,zPSFFlux,zPSFFluxErr,zKronFlux,zKronFluxErr,yPSFFlux,yPSFFluxErr,yKronFlux,yKronFluxErr
108.4654289057925,47.68652981653164,0.0007394740241579711,1.2212799447297584e-06,0.0007061050273478031,1.1322199497953989e-06,0.0012014700332656503,1.8061799664792488e-06,0.0011536399833858013,1.8802800241246587e-06,0.001429000054486096,1.6538399449927965e-06,0.0013883699430152774,1.6476200244142092e-06,0.

In [8]:
tab = ascii.read(results)

In [10]:
data = pd.DataFrame(tab)

ValueError: DataFrame constructor not properly called!

In [11]:
len(tab)

9

In [28]:
params = tab[0].colnames

In [30]:
data = pd.DataFrame([])
for p in params:
    data[p] = tab[p]

In [29]:
tab[params[0]][:5]

108.4654289057925
108.4429704462418
108.47501337931786
108.46854741195527
108.47790313867331


In [31]:
data.head()

,l,b,gPSFFlux,gPSFFluxErr,gKronFlux,gKronFluxErr,rPSFFlux,rPSFFluxErr,rKronFlux,rKronFluxErr,...,iKronFlux,iKronFluxErr,zPSFFlux,zPSFFluxErr,zKronFlux,zKronFluxErr,yPSFFlux,yPSFFluxErr,yKronFlux,yKronFluxErr
0,108.465429,47.686530,0.000739,1.221280e-06,0.000706,1.132220e-06,0.001201,1.806180e-06,0.001154,1.880280e-06,...,0.001388,1.647620e-06,0.001414,0.000003,0.001446,0.000003,0.001661,0.000004,0.001560,0.000006
1,108.442970,47.672097,0.000189,9.907360e-07,0.000179,1.144060e-06,0.000266,1.518300e-06,0.000214,1.912290e-06,...,0.000231,7.457370e-07,0.000244,0.000002,0.000229,0.000002,0.000230,0.000003,0.000225,0.000005
2,108.475013,47.683898,0.000002,4.026190e-07,0.000003,5.350150e-07,0.000003,5.439310e-07,0.000004,1.010830e-06,...,0.000005,5.180870e-07,0.000004,0.000001,-999.000000,-999.000000,0.000009,0.000003,-999.000000,-999.000000
3,108.468547,47.682417,0.000012,4.276820e-07,0.000010,5.426800e-07,0.000016,5.697600e-07,0.000014,8.067800e-07,...,0.000016,4.978740e-07,0.000020,0.000001,0.000016,0.000002,0.000021,0.000003,0.000015,0.000003
4,108.477903,47.676089,0.000004,4.023130e-07,0.000005,5.244050e-07,0.000007,5.538210e-07,0.000007,7.259320e-07,...,0.000017,6.512030e-07,0.000015,0.000001,0.000019,0.000002,0.000017,0.000003,0.000041,0.000005


In [32]:
def get_one_patch(ra, dec, radius):
    constraints = {'nDetections.gt':1}

    columns = """l,b,
    gPSFFlux,gPSFFluxErr,gKronFlux, gKronFluxErr,
    rPSFFlux, rPSFFluxErr, rKronFlux, rKronFluxErr,
    iPSFFlux, iPSFFluxErr, iKronFlux, iKronFluxErr,
    zPSFFlux, zPSFFluxErr, zKronFlux, zKronFluxErr,
    yPSFFlux, yPSFFluxErr, yKronFlux, yKronFluxErr""".split(',')
    columns = [x.strip() for x in columns]
    columns = [x for x in columns if x and not x.startswith('#')]
    results = ps1cone(ra,dec,radius,release='dr2',columns=columns,verbose=True,**constraints, table='stack')
    lines = results.split('\n')
    
    
    tab = ascii.read(results)
    
    params = tab[0].colnames
    data = pd.DataFrame([])
    for p in params:
        data[p] = tab[p]
    
    return data